Effective areas for IceCube Realtime Track Neutrino alerts

Effective areas are a useful quantity for categorizing the overall sensitivity of a neutrino detector like IceCube, including triggering and event selection effects.  Given it's position at the South Pole, all variation in sensitivity can be quanitified in neutrino energy and declination.

Additional information on calculation and use of effective areas can be found:
"All-sky Search for Time-integrated Neutrino Emission from Astrophysical Sources with 7 yr of IceCube Data"
Citation: M. G. Aartsen et al 2017 ApJ 835 151
DOI: 10.3847/1538-4357/835/2/151

We quantify the effective area separately for each of the IceCube internal event selections that contribute to the effective area. Details on each of these selection is in the supporting catalog paper:
* Effa_gfugold.txt - GFU Gold seletion
* Effa_ehegold.txt - EHE Gold selection
* Effa_hesegold.txt - HESE Gold selection
* Effa_gfubronze.txt - GFU Bronze selection
* Effa_hesebronze.txt - HESE Bronze selection

As the combination of these alerts with properly accounting for overlap of events in selections, we also provide the effective areas for the combined selection at the Gold level selections and the Gold + Bronze level selections, which serve as good representations of the overall sensitivty of the alert selections:
* Effa_all_streams_gold_only.txt - Overall GOLD selection effective areas
* Effa_all_streams_gold_bronze.txt - Overall GOLD+BRONZE selection effective areas

These effective areas are binned in neutrino energy (10 TeV - 500 PeV) and in broad declination (stored internally in the files as local zenith angle) bins that reflect the major changes in effective area due to background rejection in the down-going direction and Earth absorption at higher energies for the up-going directions.
* Declination boundaries (90 deg, 30 deg, 0 deg, -5 deg, -30 deg, -90 deg)

Plot effective areas.

In [ ]:
import numpy as np
import matplotlib as mpl
mpl.rc('font',family='serif',size=16)
mpl.rc("text",usetex = True)
import matplotlib.pyplot as plt

#Generate Declination bins
bins_coszen = [180, 120, 90, 85, 60, 0]
coszen_min = bins_coszen[:-1]
coszen_max = bins_coszen[1:]
cmin = [a-90 for a in coszen_min]
cmax = [a-90 for a in coszen_max]

color_list = ['b', 'y', 'k', 'r', 'c']
#Read effective area
effA = np.genfromtxt('eff_area/Effa_all_streams_gold_bronze.txt')
#effA = np.genfromtxt('eff_area/Effa_all_streams_gold_only.txt')


ebins = effA[:,0]

plt.figure(figsize = (8,8))
ax1 = plt.subplot (1, 1, 1)

for i,mx,mn in zip(range(1,6),cmax,cmin):
    eA = effA[:,i]
    ax1.plot(ebins,eA,drawstyle='steps-post', color=color_list[i-1], linewidth=2.5,
                  label=r'${:.2f}^\circ \leq \delta  < {:.2f}^\circ$'.format((mx), (mn)))

# Plot the overall stuff
ax1.set_ylim(1e-1,5e4) 
ax1.set_xlim(1e4, 5e8)

#plt.grid(True)
ax1.set_yscale('log')
ax1.set_xscale('log')
ax1.set_ylabel(r'Effective Area (m$^{2}$)')
ax1.set_xlabel('Neutrino Energy (GeV)')

ax1.tick_params(axis='y', which='major', labelsize=16, length = 16, width = 1, direction = 'in', right = 'on', top = 'on')
ax1.tick_params(axis='x', which='major', labelsize=16, length = 16, width = 1, direction = 'in', right = 'on', top = 'on', pad = 7)
ax1.tick_params(axis='both', which='minor', labelsize=10, length = 8, width = 1, direction = 'in', right = 'on', top = 'on')
ax1.legend(loc='upper left', ncol=1, fancybox=True)

# Set grid lines
for ymaj in ax1.yaxis.get_majorticklocs():
        ax1.axhline (y=ymaj,ls=':', color='gray', alpha =0.75, linewidth=0.5)
for xmaj in ax1.xaxis.get_majorticklocs():
        ax1.axvline (x=xmaj,ls=':', color='gray', alpha =0.75, linewidth=0.5)

plt.title('IceCube Track Alerts Gold + Bronze Effective Area')
#plt.title('IceCube Track Alerts Gold Effective Area')
plt.show()

Use the effective area to estimate the number of events from the entire sky.

To do this, the effective area over the 5 declination bands is flattened to a single all-sky declination band using
the relative area of each band as a weight.

Then a power-law astrophysical flux is convolved with the effective area to generate number of alerts expected per year

In [ ]:
import numpy as np

# Astrophysical muon neutrino flux found in:  https://iopscience.iop.org/article/10.3847/1538-4357/ac4d29
index = -2.37
flux =1.44e-18

# Astrophysical muon neutrino flux found in: https://gcn.gsfc.nasa.gov/doc/IceCube_High_Energy_Neutrino_Track_Alerts_v2.pdf
#index = -2.19
#flux =0.9e-18


days = 365.25


Afile = np.genfromtxt('eff_area/Effa_all_streams_gold_bronze.txt')
#Afile = np.genfromtxt('eff_area/Effa_all_streams_gold_only.txt')
energy_bins = Afile[:,0]


#flatten the A_eff from mutiple zenith bands
bin_zenith_bound = [180, 120, 90, 85, 60, 0]
bin_coszen_bound = np.cos(np.deg2rad(bin_zenith_bound))
#print (bin_coszen_bound)
A_eff_bins = Afile[:,1:]
#print(A_eff_bins)
all_zen_aeff = []
for e_bin in range(0,len(energy_bins)):
    this_bin_aeff = 0
    norm=0
    for item in range(1,len(bin_coszen_bound)):
        this_bin_aeff += A_eff_bins[e_bin][item-1]*(bin_coszen_bound[item]-bin_coszen_bound[item-1])
        norm += (bin_coszen_bound[item]-bin_coszen_bound[item-1])
    all_zen_aeff.append(this_bin_aeff/norm)


nus = 0 
for i in range(1,len(energy_bins)): 
    #print (all_zen_aeff[i], energy_bins[i])
    e_bin_width = energy_bins[i] - energy_bins[i-1] 
    nus += e_bin_width*all_zen_aeff[i] * 10000 * 86400 * days * 4*np.pi*flux *(energy_bins[i]/100000)**index
print(nus)